In [18]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
import pytz
from datetime import datetime
from IPython.display import display, Markdown

In [19]:
dataset = pd.read_csv("Final_Dataset.csv")

# Prepare features and target variable
X = dataset.drop(columns=['sl'])  # Features
y = dataset['sl']  # Target variable


In [3]:
ideal_ranges = {
    'Snoring Rate': (0, 100),
    'Respiration Rate': (0, 50),
    'Body Temperature': (96, 100),
    'Limb Movement': (0, 20),
    'Blood Oxygen': (0, 100),
    'Eye Movement': (0, 100),
    'Sleeping Hours': (0, 24),
    'Heart Rate': (0, 200)
}


In [20]:
scaler = MinMaxScaler(feature_range=(0, 1))  # Scales to [0, 1] by default
X = scaler.fit_transform(X)
X = X.reshape(X.shape[0], 1, X.shape[1]) 
print(X)

[[[0.88727273 0.69142857 0.48857143 ... 0.88       0.20444444 0.69142857]]

 [[0.848      0.65028571 0.468      ... 0.864      0.17244444 0.65028571]]

 [[0.27272727 0.28571429 0.78571429 ... 0.55555556 0.77777778 0.28571429]]

 ...

 [[0.95461818 0.82171429 0.13428571 ... 0.93066667 0.         0.82171429]]

 [[0.248      0.26628571 0.76628571 ... 0.54044444 0.74755556 0.26628571]]

 [[0.52581818 0.38514286 0.59942857 ... 0.71022222 0.45422222 0.38514286]]]


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
print(X_train[0])

[[0.02254545 0.03542857 0.83885714 0.06613333 0.89973333 0.11022222
  0.83288889 0.03542857]]


In [6]:
if not os.path.isfile("trained_model.h5"):
    model = Sequential([
        LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    model.fit(X_train, y_train, epochs=60, batch_size=32, validation_data=(X_test, y_test))
    model.save("trained_model.h5")
else:
    model = load_model("trained_model.h5")

In [7]:
test_loss, test_mae = model.evaluate(X_test, y_test)
display(Markdown(f'Test MAE: {test_mae:.4f}'))

4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - loss: 0.0018 - mean_absolute_error: 0.0360


Test MAE: 0.0365

In [ ]:
single_input = np.array([[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]]) 
single_input_reshaped = single_input.reshape(1, 1, 8)